In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32)))

from keras.applications import Xception
from keras import models
from keras import layers
from keras import optimizers

import numpy as np
import gc

"""
./format_data/train/
./format_data/val/
"""
train_dir = "./format_data/train/"
test_dir = "./format_data/val/"

feature_size = 299
batch_size = 5
middle_feature_size = (10,10,2048)
middle_sample_size = 3000

In [7]:
train_gen = ImageDataGenerator(rescale=1./255,rotation_range=180,horizontal_flip=True,vertical_flip=True)
test_gen = ImageDataGenerator(rescale=1./255)

train_gener = train_gen.flow_from_directory(train_dir,target_size=(feature_size,feature_size),batch_size=batch_size,shuffle = True,seed = 100,class_mode="categorical")
test_gener = test_gen.flow_from_directory(test_dir,target_size=(feature_size,feature_size),batch_size=batch_size,class_mode="categorical")

for data_batch,label_batch in train_gener:
    print(data_batch.shape)
    print(label_batch.shape)
    break

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [ ]:
X_model = Xception(weights=None,include_top=True,input_shape=(feature_size,feature_size,3),classes = 12)
#X_model.load_weights("./xception_weights_notop.h5")
X_model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])
history = model.fit_generator(train_gener,steps_per_epoch=150,epochs=5,validation_data=test_gener,validation_steps=20)

In [ ]:
model.save('Xception_whole.h5')